Here, I take a bunch of 8D L3ms out of a set of experiments that included individual ancillary noise, as well as sensor noise experiments, and reorganize the data into Rrs_unc files, 1 per band, with dimensions lat, lon and experiment. The dataset name will be rrs_unc. I also aggregate the data onto which noise was added into a separate file.

In [1]:
import xarray as xr
import os
import glob

/accounts/ekarakoy/anaconda3/lib/python3.5/site-packages/xarray/core/formatting.py:16: FutureWarning: The pandas.tslib module is deprecated and will be removed in a future version.
  from pandas.tslib import OutOfBoundsDatetime


In [6]:
from IPython.core.display import display, HTML
display(HTML("<style>.container {width:90% !important;}</style>"))

In [4]:
dataDir = '/accounts/ekarakoy/disk02/UNCERTAINTIES/Monte-Carlo/DATA/AncillaryMC/'
outDir = 'Synthesis'
expDirs = [d for d in os.listdir(dataDir) if outDir not in d]
fpattern = 'S2003193*.L3m_8D*.nc'
fpaths = [glob.glob(os.path.join(dataDir, expDir, fpattern))[0] for expDir in expDirs]

In [5]:
expDirs

['O3', 'Pressure', 'WindSpeed', 'Lt', 'All', 'RH']

In [7]:
fpaths

['/accounts/ekarakoy/disk02/UNCERTAINTIES/Monte-Carlo/DATA/AncillaryMC/O3/S20031932003200.L3m_8D_O3.nc',
 '/accounts/ekarakoy/disk02/UNCERTAINTIES/Monte-Carlo/DATA/AncillaryMC/Pressure/S20031932003200.L3m_8D_Pressure.nc',
 '/accounts/ekarakoy/disk02/UNCERTAINTIES/Monte-Carlo/DATA/AncillaryMC/WindSpeed/S20031932003200.L3m_8D_WindSpeed.nc',
 '/accounts/ekarakoy/disk02/UNCERTAINTIES/Monte-Carlo/DATA/AncillaryMC/Lt/S20031932003200.L3m_8D_Lt.nc',
 '/accounts/ekarakoy/disk02/UNCERTAINTIES/Monte-Carlo/DATA/AncillaryMC/All/S20031932003200.L3m_8D_All.nc',
 '/accounts/ekarakoy/disk02/UNCERTAINTIES/Monte-Carlo/DATA/AncillaryMC/RH/S20031932003200.L3m_8D_RH.nc']

In [8]:
bands = [412, 443, 490, 510, 555, 670]

In [15]:
with xr.open_mfdataset(fpaths, concat_dim='experiment') as allData:
    allData.coords['experiment'] = expDirs
    # rrs unc section
    for band in bands:
        foutpath = os.path.join(dataDir, outDir, '%s%d%s' %(fpaths[0].rsplit('/')[-1].split('8D')[0],
                                                           band, '_8D.nc'))
        if not os.path.exists(os.path.dirname(foutpath)):
            os.makedirs(os.path.dirname(foutpath))
        data = allData.data_vars['Rrs_unc_%d' % band]
        data.name = 'rrs_unc'
        dsData = data.to_dataset()
        dsData.to_netcdf(path=foutpath, engine='netcdf4')